In [183]:
import os
import numpy as np
import re
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from itertools import combinations
import string
from collections import Counter, defaultdict

In [184]:
path_connected_data = "C:\\Users\\tykun\\OneDrive\\Documents\\SchoolDocs\\VSCodeProjects\\connectedData\\"
path_temp_data = "C:\\Users\\tykun\\OneDrive\\Documents\\SchoolDocs\\VSCodeProjects\\connectedData\\temporaryData\\"
# year = "2009"

# full_path = f"{path_connected_data}{year}_gptDataframe.csv"
# grouped_path = f"{path_temp_data}{year}_grouped.csv"
# system_path = f"{path_temp_data}{year}_state_systems.csv"
affiliation_path = f"{path_connected_data}affiliation.csv"
state_system_path = f"{path_temp_data}state_systems.csv"

In [185]:
# try:
#     stopwords.words('english')
#     nltk.data.find('tokenizers/punkt')
# except (LookupError, OSError):
#     nltk.download('stopwords')
#     nltk.download('punkt')
# stopwords = set(stopwords.words('english'))

In [186]:
STATE_SYSTEMS = { 
           "University of Alabama System": ["University of Alabama at Birmingham", "University of Alabama in Huntsville", "University of Alabama - Tuscaloosa", "The University of Alabama"], 
           "University of Alaska System": ["University of Alaska Anchorage", "University of Alaska Fairbanks", "University of Alaska Southeast"], 
           "Arizona Board of Regents": ["Arizona State University", "Northern Arizona University", "University of Arizona"], 
           "University of Arkansas System": ["University of Arkansas at Fayetteville", "University of Arkansas at Little Rock", "University of Arkansas for Medical Sciences", "University of Arkansas at Monticello", "University of Arkansas at Pine Bluff"], 
           "University of California System": ["University of California Berkeley", "University of California Davis", "University of California Irvine", "University of California Los Angeles", "University of California Riverside", 
                                               "University of California San Diego", "University of California San Francisco", "University of California Santa Barbara", "University of California Santa Cruz", "University of California Merced"],
           "California State University System": ["California Maritime Academy", "California Polytechnic State University, San Luis Obispo", "California State Polytechnic University, Pomona", "California State University, Bakersfield",  
                                                  "California State University Channel Islands", "California State University, Chico", "California State University, Dominguez Hills", "California State University, East Bay", "California State University, Fresno", 
                                                  "California State University, Fullerton", "California State University, Long Beach", "California State", "California State University, Monterey Bay", "California State University, Northridge", "California State University, Sacramento", 
                                                  "California State University, San Bernardino", "California State University, San Marcos", "California State University, Stanislaus", "Humboldt State University", "San Diego State University", "San Diego State University, Imperial Valley Campus", "San Francisco State University", "San Jose State University", "Sonoma State University", "Imperial Valley Campus"], 
           "University of Colorado System": ["University of Colorado at Boulder", "University Of Colorado at Colorado Springs", "University of Colorado Denver"], 
           "Connecticut": [], 
           "Delaware": [], 
           #not in 2009
           "Florida State University System": ["Florida Agricultural and Mechanical University", "Florida Atlantic University", "Florida Gulf Coast University", "Florida International University", "Florida State University", "University of Central Florida", "University of Florida", "University of North Florida", "University of South Florida", "University of West Florida"], 
           "University System of Georgia": ["Augusta State University", "Georgia Institute of Technology", "Georgia Southern University", "Georgia Southwestern State University", "Georgia State University", "Kennesaw State University", 
                                            "Medical College of Georgia", "North Georgia College and State University", "Southern Polytechnic State University", "State University of West Georgia", "The University of Georgia", "University of West Georgia"], 
           "University of Hawaii System": ["University of Hawaii at Hilo", "University of Hawaii Manoa", "University of Hawaii - West Oahu"], 
           "Illinois": [], 
           "Indiana University System": ["Indiana University Bloomington", "Indiana University East", "Indiana University Kokomo", "Indiana University Northwest", "Indiana University - Purdue University Indianapolis", "Indiana University South Bend", "Indiana University Southeast", "Indiana University-Purdue University A Indianapolis", "Indiana University-Purdue University A Indianapolis [Iupui]"], 
           #Iowa and Kansas board not in 2009 dataframe
           "Iowa Board of Regents": ["Iowa State University", "University of Northern Iowa", "University of Iowa"], 
           "Kansas Board of Regents": ["Emporia State University", "Kansas State University", "University of Kansas", "Wichita State University", "Fort Hays State University", "Pittsburg State University"], 
           "Kentucky": [], 
           "Louisiana State University System": ["Louisiana State University", "Louisiana State University at Alexandria", "Louisiana State University Eunice", "Louisiana State University Health Sciences Center New Orleans", "Louisiana State University In Shreveport", "University of New Orleans", "Louisiana State University And Agricultural And Mechanical College", "Louisiana State University At Eunice"],
           "University of Louisiana System": ["Grambling State University", "Louisiana Tech University", "McNeese State University", "Nicholls State University", "Northwestern State University of Louisiana", "Southeastern Louisiana University", "University of Louisiana at Lafayette", "University of Louisiana at Monroe"], 
           "University of Maine System": ["University of Maine Orono", "University of Maine at Augusta", "University of Maine at Farmington", "University of Maine at Fort Kent", "University of Maine at Machias", "University of Maine at Presque Isle", "University of Southern Maine"], 
           "University System of Maryland": ["Bowie State University", "Coppin State College", "Coppin State University", "Frostburg State University", "Salisbury State University", "Salisbury University", "Towson University", "University of Baltimore", "University of Maryland, Baltimore", "University of Maryland, Baltimore County", "University of Maryland, College Park", 
                                             "University of Maryland, Eastern Shore", "University of Maryland University College"], 
           "University of Massachusetts System": ["University of Massachusetts Amherst", "University of Massachusetts Boston", "University of Massachusetts Dartmouth", "University of Massachusetts at Lowell", "University of Massachusetts Worcester"], 
           "Michigan": [], 
           "Minnesota State Colleges and Universities System": ["Minnesota State University, Mankato", "Minnesota State University Moorhead"], 
           "Mississippi Institutions of Higher Learning": ["Alcorn State University", "Delta State University", "Jackson State University", "Mississippi State University", "Mississippi University for Women", "Mississippi Valley State University", "University of Mississippi", "University of Mississippi Medical Center", "University of Southern Mississippi"], 
           "University of Missouri System": ["University of Missouri, Columbia", "University of Missouri - Kansas City", "Missouri University of Science and Technology", "University of Missouri - St. Louis", "University Of Missouri St Louis"], 
           "Montana University System": ["Montana State University", "Montana State University-Billings", "Montana State University-Northern", "Montana Tech of The University of Montana", "University of Montana-Missoula", "University Of Montana Missoula", "University of Montana Western"], 
           "University of Nebraska System": ["University of Nebraska - Lincoln", "University Of Nebraska Lincoln", "University of Nebraska at Kearney", "University of Nebraska at Omaha", "University of Nebraska Medical Center"], 
           "Nevada System of Higher Education": ["University of Nevada, Las Vegas", "University of Nevada, Reno"], 
           "University System of New Hampshire": ["University of New Hampshire", "University of New Hampshire-Manchester", "Keene State College", "Plymouth State College", "Plymouth State University"], 
           "New Jersey": ["Rutgers State University of New Jersey", "Rutgers State University of New Jersey - Camden", "Rutgers State University of New Jersey - New Brunswick", "Rutgers State University of New Jersey - Newark"], 
           "New Mexico": [], 
           "State University of New York System": ["University at Albany, SUNY", "University at Albany", "State University of New York at Binghamton", "State University of New York at Brockport", "Buffalo State College", "University at Buffalo, State University of New York", "State University of New York College at Cortland", "SUNY Downstate Medical Center", 
                                                   "Empire State College", "State University of New York at Fredonia", "State University of New York at Geneseo", "State University of New York at New Paltz", "State University of New York at Old Westbury", "State University of New York College at Oneonta", "State University Of New York At Oneonta", "State University of New York at Oswego", "State University of New York College at Plattsburgh", "State University Of New York At Plattsburgh", "State University of New York at Potsdam", "State University of New York, Stony Brook University", "Upstate Medical University",
                                                   "State University Of New York, Stony Brook", "State University Of New York"], 
           "City University of New York System": ["Bernard M. Baruch College", "Brooklyn College", "City College of New York", "College of Staten Island", "Medgar Evers College", "Hunter College", "John Jay College of Criminal Justice", "Herbert H. Lehman College", "New York City College of Technology", "Queens College", "York College",
                                                  "Herbert H Lehman College"],
           "University of North Carolina System": ["Appalachian State University", "East Carolina University", "Elizabeth City State University", "Fayetteville State University", "North Carolina Agricultural and Technical State University", "North Carolina Central University", "North Carolina School of the Arts", "North Carolina State University",
                                                    "University of North Carolina at Asheville", "University of North Carolina at Chapel Hill", "University of North Carolina at Charlotte", "University of North Carolina at Greensboro", "University of North Carolina at Pembroke", "University of North Carolina at Wilmington", "Western Carolina University", "Winston-Salem State University"], 
           "North Dakota University System": ["Dickinson State University", "Mayville State University", "Minot State University", "North Dakota State University", "University of North Dakota", "Valley City State University"], 
           #Not in 2009
           "University System of Ohio": ["Bowling Green State University", "Central State University", "Cleveland State University", "Kent State University", "Miami University", "Northeast Ohio Medical University", "Ohio State University", "Ohio University", "University of Akron", "University of Cincinnati", "University of Toledo", "Wright State University"], 
           #Not in 2009
           "Oklahoma State System": ["Oklahoma State University", "University of Oklahoma", "University of Oklahoma Health Sciences Center", "University of Oklahoma - Tulsa", "University of Oklahoma, Health Sciences Center", "University of Oklahoma-Tulsa"], 
           "Oregon University System": ["Eastern Oregon University", "Oregon Institute of Technology", "Oregon State University", "Portland State University", "Southern Oregon University", "University of Oregon", "Western Oregon University"], 
           "Pennsylvania State System": ["Bloomburg University of Pennsylvania", "California University of Pennsylvania", "Cheyney University of Pennsylvania", "Clarion University of Pennsylvania", "East Stroudsburg University of Pennsylvania", "Edinboro University of Pennsylvania", 
                                        "Indiana University of Pennsylvania", "Kutztown University Of Pennsylvania", "Lock Haven University of Pennsylvania", "Mansfield University of Pennsylvania", "Millersville University of Pennsylvania", "Shippensburg University of Pennsylvania", 
                                        "Slippery Rock University of Pennsylvania", "West Chester University of Pennsylvania", "Pennsylvania State University", "Penn State Milton S. Hershey Medical Center", "Pennsylvania State University - Abington College", "Pennsylvania State University - Altoona College", 
                                        "Pennsylvania State University - Berks College", "Pennsylvania State University - Great Valley", "Pennsylvania State University - Harrisburg", "Pennsylvania State University at Erie - The Behrend College", "Pennsylvania State University at Schuylkill"],
           "Rhode Island": [], 
           "South Carolina": [], 
           #not in 2009
           "South Dakota University System": ["South Dakota State University", "University of South Dakota"],
           "University of Tennessee System": ["University of Tennessee at Chattanooga", "University of Tennessee at Knoxville", "University of Tennessee at Martin", "University of Tennessee Health Science Center"], 
           "Tennessee Board of Regents": ["Austin Peay State University", "East Tennessee State University", "Middle Tennessee State University", "Tennessee State University", "Tennessee Technology University", "University of Memphis", "Tennessee Technological University"],
           "University of Texas System": ["University of Texas at Arlington", "University of Texas at Austin", "University of Texas at Brownsville/Texas Southmost College", "University of Texas at Dallas", "University of Texas at El Paso", "University of Texas - Pan American", "University of Texas of the Permian Basin", 
                                          "University of Texas at San Antonio", "University of Texas at Tyler", "University of Texas Southwestern Medical Center at Dallas", "University of Texas Health Science Center at Houston", "University of Texas Health Science Center at San Antonio", "University of Texas Medical Branch at Galveston"],
           "Texas A&M System": ["Health Science Center", "Prairie View A&M University", "Tarleton State University", "Texas A&M International University", "Texas A&M University - College Station", "Texas A&M University - Commerce", "Texas A&M University - Corpus Christi", "Texas A&M University at Galveston", "Texas A&M University - Kingsville", "Texas A&M University - Texarkana", "West Texas A&M University"], 
           "Texas State System": ["Lamar Institute of Technology", "Lamar State College - Orange", "Lamar State College - Port Arthur", "Lamar University", "Sam Houston State University", "Sul Ross State University", "Texas State University - San Marcos"],
           "Texas Tech University System": ["Angelo State University", "Texas Tech University", "Texas Tech University Health Sciences Center", "Texas Tech University Health Sciences Center at Amarillo", "Texas Tech University Health Sciences Center at El Paso", "Texas Tech University Health Sciences Center at the Permian Basin"],
           "University of Houston System": ["University of Houston", "University of Houston - Clear Lake", "University of Houston - Downtown", "University of Houston - Victoria"],
           "University of North Texas System": ["University of North Texas", "University of North Texas Dallas Campus", "University Of North Texas At Dallas", "University of North Texas Health Science Center at Fort Worth"],
           #Not in 2009
           "Utah University System": ["University of Utah", "Utah State University", "Weber State University"], 
           "Vermont": [], 
           "Virginia": [], 
           "Washington": [], 
           "West Virginia": [], 
           "University of Wisconsin System": ["University of Wisconsin - Eau Claire", "University of Wisconsin - Green Bay", "University of Wisconsin - La Crosse", "University of Wisconsin - Madison", "University of Wisconsin - Milwaukee", "University of Wisconsin - Oshkosh", "University of Wisconsin - Parkside", 
                                              "University of Wisconsin - Platteville", "University of Wisconsin - River Falls", "University of Wisconsin - Stevens Point", "University of Wisconsin - Stout", "University of Wisconsin - Superior", "University of Wisconsin - Whitewater"], 
           "Wyoming": [] }

In [187]:
#Extract the names of all the institutions for validation
def extract_institutions(df):
    institution_list = []
    for index, row in df.iterrows():
        if row["Institution"] not in institution_list:
            institution_list.append(row["Institution"])
    return institution_list

def extract_subinstitutions(df):
    institution_list = []
    for index, row in df.iterrows():
        if row["SubInstitution"] not in institution_list:
            institution_list.append(row["Institution"])
    return institution_list

In [188]:
def exact_matches(df, institution_list):
    match_list = []
    for key, value in STATE_SYSTEMS.items():
        for university in value:
            if university.title() in institution_list:
                df.loc[df['Institution'] == university, "StateSystem"] =  key
                match_list.append(str(university))
    return df, match_list

def remove_punct_stopwords(element):
    words = word_tokenize(element)
    cleaned = [s for s in words if s.lower() not in stopwords and s not in string.punctuation]
    return ' '.join(cleaned)
    
def stopword_matches(df, institution_list):
    match_list = []
    cleaned_institution_dict = {s: remove_punct_stopwords(s) for s in institution_list}
    for key, value in STATE_SYSTEMS.items():
        for university in value:
            cleaned_university = remove_punct_stopwords(university).title()
            # Find the original institution name that corresponds to the cleaned name
            original_institution = next((orig for orig, cleaned in cleaned_institution_dict.items() if cleaned == cleaned_university), None)
            if original_institution:
                df.loc[df['Institution'] == original_institution, "StateSystem"] = key
                match_list.append(str(original_institution))
    return df, match_list

def remove_dash_spacing(s):
    return re.sub(r'\s*-\s*', '-', s)

def remove_dash(s):
    return re.sub(r'\s*-\s*', ' ', s)

def extra_dash_spacing(df, institution_list):
    match_list = []
    for key, value in STATE_SYSTEMS.items():
        for university in value:
            cleaned_university = remove_dash_spacing(university)
            removed_dash_university = remove_dash(university)
            if ("-" in cleaned_university and cleaned_university.title() in institution_list):
                df.loc[df['Institution'] == cleaned_university.title(), "StateSystem"] =  key
                match_list.append(str(cleaned_university.title()))
            elif (removed_dash_university.title() in institution_list):
                df.loc[df['Institution'] == removed_dash_university.title(), "StateSystem"] =  key
                match_list.append(str(removed_dash_university.title()))
    return df, match_list

def subinstitution_match(df, subinstitution_list):
    match_list = []
    for key, value in STATE_SYSTEMS.items():
        for university in value:
            if university.title() in subinstitution_list:
                df.loc[df['Institution'] == university, "StateSystem"] =  key
                match_list.append(str(university))
    return df, match_list

def mark_systems(full_dataframe, match_list):
    full_dataframe["State System"] = "False"
    match_list_title = [s.title() for s in match_list]
    for index, row in full_dataframe.iterrows():
        institution = row["Institution"].title()
        subinstitution = row["SubInstitution"]
        if institution in match_list_title or subinstitution in match_list_title:
            full_dataframe.at[index, "State System"] = "True"
        elif "System" in institution:
            full_dataframe.at[index, "State System"] = "True"
    return full_dataframe


In [189]:
# full_dataframe = pd.read_csv(full_path)
# state_systems_df = full_dataframe[["Institution"]].drop_duplicates()
# state_systems_df["StateSystem"] = False

# institution_list_pre = extract_institutions(full_dataframe)
# subinstitution_list_pre = extract_subinstitutions(full_dataframe)

# #convert list to titlecase
# institution_list = [s.title() for s in institution_list_pre]
# subinstitution_list = [s.title() for s in subinstitution_list_pre]

# state_systems_df, exact_match= exact_matches(state_systems_df, institution_list)
# state_systems_df, stop_match = stopword_matches(state_systems_df, institution_list)
# state_systems_df, dash_match = extra_dash_spacing(state_systems_df, institution_list)
# state_systems_df, sub_match = subinstitution_match(state_systems_df, subinstitution_list)

# all_matched_systems = list(set(exact_match + stop_match + dash_match + sub_match))
# all_matched_systems_title = [s.title() for s in all_matched_systems]

In [190]:
# state_systems_df = state_systems_df.merge(full_dataframe[['Institution', 'AffiliationId']], on='Institution', how='left')
# state_systems_df = state_systems_df.drop_duplicates()
# state_systems_df.to_csv(system_path, index = False)

In [191]:
all_existing_systems = [element for values in STATE_SYSTEMS.values() for element in values]

In [192]:
affiliation_df = pd.read_csv(affiliation_path)
affiliation_df["FullNameTitle"] = affiliation_df["FullName"].str.title()
affiliation_institutions = affiliation_df["FullName"].values
affiliation_institutions_title = [s.title() for s in affiliation_institutions]
for key, value in STATE_SYSTEMS.items():
    for s in value:
        if s.title() in affiliation_institutions_title:
            affiliation_df.loc[affiliation_df["FullNameTitle"] == s.title(), "StateSystem"] = key
        
affiliation_df = affiliation_df.drop(columns=["FullNameTitle"])
affiliation_df["StateSystem"] = affiliation_df.groupby("AffiliationId")["StateSystem"].transform(lambda x: x.fillna(method='ffill').fillna(method='bfill'))
affiliation_df = affiliation_df.rename(columns={"FullName": "Institution"})
affiliation_df.to_csv(state_system_path, index=False)



C:\Users\tykun\AppData\Local\Temp\ipykernel_17116\381385247.py:11: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  affiliation_df["StateSystem"] = affiliation_df.groupby("AffiliationId")["StateSystem"].transform(lambda x: x.fillna(method='ffill').fillna(method='bfill'))
C:\Users\tykun\AppData\Local\Temp\ipykernel_17116\381385247.py:11: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  affiliation_df["StateSystem"] = affiliation_df.groupby("AffiliationId")["StateSystem"].transform(lambda x: x.fillna(method='ffill').fillna(method='bfill'))


In [193]:
matched_list = []
for index, row in affiliation_df.iterrows():
    if not pd.isna(row["StateSystem"]):
        matched_list.append(row["Institution"].title())

print(matched_list)
for s in all_existing_systems:
    if s.title() not in matched_list:
        print(s)

['Arizona State University', 'Austin Peay State University', 'Austin Pay State University', 'Bowling Green State University', 'California Maritime Academy', 'California Maritime Academy [Cal Maritime]', 'California Polytechnic State University', 'California Polytechnic State University, San Luis Obispo', 'California Polytechnic State University San Luis Obispo', 'California Polytechnic State University, San Luis Obispo [Cal Poly]', 'California State Polytechnic University', 'California State Polytechnic University, Pomona', 'California State Polytechnic University Pomona', 'California Polytechnic State University', 'California State Polytechnic University', 'California State University, Bakersfield', 'California State University Bakersfield', 'California State University Channel Islands', 'California State University, Channel Islands', 'California State University, Chico', 'California State University Chico', 'California State University, Dominguez Hills', 'California State University 